In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading data

In [ ]:
import pandas as pd
import numpy as np
train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
train.drop(['Id'], axis=1, inplace=True)
train

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')
test.drop(['Id'], axis=1, inplace=True)
test

# Data processing

In [ ]:
train.loc[train['Cover_Type'] > 4] -= 1
train['Cover_Type'] -= 1
train.drop(3403875, inplace=True)
train = train.reset_index()

In [ ]:
train.drop(['index'], axis=1, inplace=True)

In [ ]:
train.Cover_Type.value_counts()

In [ ]:
train.describe()

In [ ]:
train.columns

## Processing highly correlated features

In [ ]:
corr_matrix = train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),
k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]


In [ ]:
train = train.drop(to_drop, axis=1)

In [ ]:
to_drop

In [ ]:
test = test.drop(to_drop, axis=1)

## Correlation matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize = (15,10))

sns.set(font_scale=0.4)
corr_matrix = train.corr()
sns.heatmap(corr_matrix, annot=True, linewidths=.1, cmap='coolwarm')
plt.title('Correlation matrix')
plt.show()

## Balancing data

In [ ]:
temp = train.loc[train.Cover_Type == 1]
temp = temp.append(train.loc[train.Cover_Type == 0])
temp

In [ ]:
temp = temp.sample(frac=1)
temp

In [ ]:
temp = temp[:400000][:]
temp

In [ ]:
temp.Cover_Type.value_counts()

In [ ]:
train = train.drop(train.loc[train.Cover_Type == 1].index)
train = train.drop(train.loc[train.Cover_Type == 0].index)
train = train.append(temp)
train.Cover_Type.value_counts()

In [ ]:
#train['Cover_Type'] = np.int64(train['Cover_Type'])

In [ ]:
temp = None

In [ ]:
train

In [ ]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0, k_neighbors=5)

# вектор признаков
X_temp = train.drop(['Cover_Type'], axis=1) 
# вектор целевой переменной
y_temp = train['Cover_Type']

column = X_temp.columns

X_temp, y_temp = os.fit_resample(X_temp, y_temp)
X_temp = pd.DataFrame(data=X_temp, columns=column)
y_temp = pd.DataFrame(data=y_temp, columns=['Cover_Type'])

In [ ]:
X = train.drop(['Cover_Type'], axis=1)
y = train['Cover_Type']
X = X.append(X_temp)
y = y.to_numpy()
y = pd.DataFrame(y, columns=['Cover_Type'])
y = y.append(y_temp)

In [ ]:
X_temp, y_temp = None, None

In [ ]:
X = X.reset_index()
X.drop(['index'], axis=1, inplace=True)
X

In [ ]:
y = y.reset_index()
y.drop(['index'], axis=1, inplace=True)
y

In [ ]:
train = pd.concat([X, y], axis=1, ignore_index=False)
train

In [ ]:
X = None
y = None

## Normalization

In [ ]:
X = train.drop(['Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40', 'Cover_Type'], axis=1)
y = train['Cover_Type']

In [ ]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X)
X_norm = scaler.transform(X)
X_norm = pd.DataFrame(X_norm, columns=X.columns)
X_norm

In [ ]:
test = scaler.transform(test)

In [ ]:
train = pd.concat([X_norm, train[['Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40', 'Cover_Type']]], axis=1, ignore_index=False)
train

In [ ]:
X_norm = None

## Разделение данных

In [ ]:
X = train.drop(['Cover_Type'], axis=1)
y = train['Cover_Type']
print(X.shape)
print(y.shape)

In [ ]:
train = None

# Neural network

In [ ]:
import torch
import random
import numpy as np

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
X_train = X.to_numpy()
X_train = torch.from_numpy(X_train).float()

In [ ]:
y_train = y.to_numpy()
y_train = torch.from_numpy(y_train)

In [ ]:
class ClassifierNet(torch.nn.Module):

    def __init__(self, n_hidden_neurons):
        super(ClassifierNet, self).__init__()

        self.fc1 = torch.nn.Linear(51, n_hidden_neurons)
        self.ac1 = torch.nn.ReLU()
        self.batch_norm1 = torch.nn.BatchNorm1d(n_hidden_neurons)

        self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.ac2 = torch.nn.ReLU()
        self.batch_norm2 = torch.nn.BatchNorm1d(n_hidden_neurons)

        self.fc3 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.ac3 = torch.nn.ReLU()
        self.batch_norm3 = torch.nn.BatchNorm1d(n_hidden_neurons)

        self.fc4 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons // 2)
        self.ac4 = torch.nn.ReLU()
        self.batch_norm4 = torch.nn.BatchNorm1d(n_hidden_neurons // 2)

        self.fc5 = torch.nn.Linear(n_hidden_neurons // 2, n_hidden_neurons // 4)
        self.ac5 = torch.nn.ReLU()
        self.dr = torch.nn.Dropout(p=0.5)
        self.batch_norm5 = torch.nn.BatchNorm1d(n_hidden_neurons // 4)

        self.fc6 = torch.nn.Linear(n_hidden_neurons // 4, 6)
    

    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        x = self.batch_norm1(x)
        x = self.fc2(x)
        x = self.ac2(x)
        x = self.batch_norm2(x)
        x = self.fc3(x)
        x = self.ac3(x)
        x = self.batch_norm3(x)
        x = self.fc4(x)
        x = self.ac4(x)
        x = self.batch_norm4(x)
        x = self.fc5(x)
        x = self.ac5(x)
        x = self.dr(x)
        x = self.batch_norm5(x)
        x = self.fc6(x)
        
        return x

net = ClassifierNet(128)

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = net.to(device) 

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=1.0e-3)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

In [ ]:
batch_size = 500000

for epoch in range(1000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        X_batch = X_train[batch_indexes].to(device)
        y_batch = y_train[batch_indexes].to(device)
        
        preds = net.forward(X_batch)
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
    if epoch % 10 == 0:
        net.eval()
        preds_val = net.forward(X_val.to(device))
        print(epoch, "train: " ,(preds.argmax(dim=1) == y_batch).float().mean())
        print(epoch, "valid: " ,(preds_val.argmax(dim=1) == y_val.to(device)).float().mean())
        net.train()

In [ ]:
X_test = test.to_numpy()
X_test = torch.from_numpy(X_test).float()
X_test = X_test.to(device)

In [ ]:
y_test = net.forward(X_test)

In [ ]:
y_test = y_test.argmax(dim=1)

In [ ]:
y_test = y_test.cpu()

In [ ]:
y_test += 1
y_test

In [ ]:
y_test[y_test > 4] += 1
y_test

In [ ]:
ind = np.linspace(4000000, 4999999,num= 1000000, dtype=int)
ind.shape

In [ ]:
y_pred = pd.DataFrame({'id': ind, 'Cover_Type': y_test})
print(y_pred.Cover_Type.value_counts())
# y_test = y_test.set_index('id')
print(y_pred)

In [ ]:
y_pred.to_csv('submission.csv', index=None)

# Model selection

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score


log = LogisticRegression()
tree = DecisionTreeClassifier()
knn = KNeighborsClassifier()
xgb = xgb.XGBClassifier()
catboost = CatBoostClassifier() 

In [ ]:
accuracy = []
X_train, X_test, y_train, y_test = train_test_split(train.drop(['Cover_Type'], axis=1), train.Cover_Type, test_size=0.3, random_state=42)

models = [tree, knn, xgb, catboost]

for model in models:
    print(model)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy.append(accuracy_score(y_test, y_pred))

In [ ]:
df_score = pd.DataFrame({'models': models, 'accuracy': accuracy})
df_score

# DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier()

In [ ]:
from sklearn.model_selection import GridSearchCV


grid = {'criterion':['gini','entropy'],
             'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50,70,90,120,150]}
clf = GridSearchCV(tree, grid, cv=5)
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
clf.fit(X=X_train, y=y_train)

In [ ]:
y_pred = clf.predict(test)
y_pred

In [ ]:
y_pred.shape

In [ ]:
ind = np.linspace(4000000, 4999999,num= 1000000, dtype=int)
ind.shape

In [ ]:
y_pred = pd.DataFrame({'id': ind, 'Cover_Type': y_pred})
print(y_pred.Cover_Type.value_counts())
# y_test = y_test.set_index('id')
print(y_pred)

In [ ]:
y_pred.loc[y_pred['Cover_Type'] > 4] += 1
y_pred['Cover_Type'] += 1

In [ ]:
y_pred.Cover_Type.value_counts()

In [ ]:
y_pred.to_csv('submission.csv', index=False)

# CatBoost

In [ ]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(task_type='GPU')

In [ ]:
grid = {'learning_rate': [0.2, 0.3, 0.4],
        'depth': [5, 7, 9],
        'l2_leaf_reg': [1, 3, 5],
        'iterations': [1000, 1200, 1300]}

grid_search_result = cat.grid_search(grid, X=X, y=y, plot=True, verbose=False)

In [ ]:
grid_search_result['params']   

In [ ]:
cat = CatBoostClassifier()

In [ ]:
cat.fit(X=features_kbest, y=train.Cover_Type, verbose=False);

In [ ]:
y_test = cat.predict(test)

In [ ]:
y_test

In [ ]:
y_test = y_test.reshape(-1)

In [ ]:
ind = np.linspace(4000000, 4999999,num= 1000000, dtype=int)
ind.shape

In [ ]:
y_pred = pd.DataFrame({'id': ind, 'Cover_Type': y_test})
print(y_pred.Cover_Type.value_counts())
# y_test = y_test.set_index('id')
print(y_pred)

In [ ]:
y_pred.loc[y_pred['Cover_Type'] > 4] += 1
y_pred['Cover_Type'] += 1

In [ ]:
y_pred.Cover_Type.value_counts()

In [ ]:
y_pred.to_csv('submission.csv', index=False)